In [10]:
## The input shape of the LSTM layer is 700 × 30, 700 × 60 or 700 × 90 
## where 700 is the amount of music
## The output of the LSTM layer is 700 × 128, 128 is the number of units in LSTM layer
## Additionally, the activation functions used in this project are the hyperbolic tangen
## and sigmoid

In [1]:
from keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.layers import LSTM
from keras.layers import Bidirectional
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import regularizers
import pandas as pd
import sklearn as skl
import sklearn.model_selection as skm
from keras.optimizers import Adam
from keras.models import Model
import numpy as np
from keras import backend as K
import import_ipynb
import SVM_KNN as sknn

importing Jupyter notebook from SVM_KNN.ipynb
Accuracy on SVM for ZCR: 23.67%
Accuracy on SVM for MFCC: 64.33%
Accuracy on SVM for ZCR and MFCC: 62.00%
Accuracy on KNN for ZCR: 19.00%
Accuracy on KNN for MFCC: 31.67%
Accuracy on KNN for ZCR and MFCC: 31.67%


In [2]:
GTZAN_DIR = '/Users/nkkaushal/Documents/Fall22/ECS_271/Project/dataset/GTZAN_data/'
features = pd.read_csv(GTZAN_DIR+"features.csv")

In [3]:
def get_test_train_split(X, Y):

    scaler = skl.preprocessing.StandardScaler(copy=False)
    scaler.fit_transform(X)
    
    X_train, X_test, Y_train, Y_test = skm.train_test_split(X, Y, test_size=0.3, random_state=42)

    # Be sure training samples are shuffled.
    X_train, Y_train = skl.utils.shuffle(X_train, Y_train, random_state=42)
    X_test, Y_test = skl.utils.shuffle(X_test, Y_test, random_state=42)

    return [X_train, X_test, Y_train, Y_test]

In [4]:
X_ZCR = features[['ZRC' + str(ind) for ind in range(1,31)]]
X_MFCC = features[['MFCC' + str(ind) for ind in range(1,61)]]
X_ZCR_MFCC = features[['ZRC' + str(ind) for ind in range(1,31)]+['MFCC' + str(ind) for ind in range(1,61)]]
Y = features['label']

genre_list = Y.unique().tolist()

In [5]:
def one_hot(Y_genre_strings):
    y_one_hot = np.zeros((Y_genre_strings.shape[0], len(genre_list)))
    for i, genre_string in enumerate(Y_genre_strings):
        index = genre_list.index(genre_string)
        y_one_hot[i, index] = 1
    return y_one_hot

In [35]:
Y_one_hot = one_hot(Y)
X_train_zcr, X_test_zcr, Y_train_zcr, Y_test_zcr = get_test_train_split(X_ZCR, Y_one_hot)
X_train_zcr = X_train_zcr.to_numpy().reshape((X_train_zcr.shape[0], X_train_zcr.shape[1], 1))
X_test_zcr = X_test_zcr.to_numpy().reshape((X_test_zcr.shape[0], X_test_zcr.shape[1], 1))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_zcr.shape[1], X_train_zcr.shape[2])

In [6]:
# Y_one_hot = one_hot(Y)
X_train_mfcc, X_test_mfcc, Y_train_mfcc, Y_test_mfcc = get_test_train_split(X_MFCC, Y)
Y_one_hot_train = one_hot(Y_train_mfcc)
Y_one_hot_test = one_hot(Y_test_mfcc)
X_train_mfcc = X_train_mfcc.to_numpy().reshape((X_train_mfcc.shape[0], X_train_mfcc.shape[1], 1))
X_test_mfcc = X_test_mfcc.to_numpy().reshape((X_test_mfcc.shape[0], X_test_mfcc.shape[1], 1))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_mfcc.shape[1], X_train_mfcc.shape[2])

In [274]:
Y_one_hot = one_hot(Y)
X_train_zcr_mfcc, X_test_zcr_mfcc, Y_train_zcr_mfcc, Y_test_zcr_mfcc = get_test_train_split(X_ZCR_MFCC, Y_one_hot)
X_train_zcr_mfcc = X_train_zcr_mfcc.to_numpy().reshape((X_train_zcr_mfcc.shape[0], 1, X_train_zcr_mfcc.shape[1]))
X_test_zcr_mfcc = X_test_zcr_mfcc.to_numpy().reshape((X_test_zcr_mfcc.shape[0], 1, X_test_zcr_mfcc.shape[1]))
# Y_train_zcr = Y_train_zcr.reshape((Y_train_zcr.shape[0], Y_train_zcr.shape[1], 1))
input_shape = (X_train_zcr_mfcc.shape[1], X_train_zcr_mfcc.shape[2])

In [110]:
Y_train_zcr.shape

(700, 10)

In [13]:
inputs = Input(input_shape)
# lstm1 = LSTM(units=64, return_sequences=True)(inputs)
lstm = LSTM(units=128, return_sequences=False, dropout=0.05, recurrent_dropout=0.35)(inputs)
# lstm1 = LSTM(units=128, activation = 'relu', return_sequences=False)(lstm)
dense = Dense(units=10, activation="softmax")(lstm)

model = Model(inputs, dense)

In [14]:
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 60, 1)]           0         
                                                                 
 lstm_3 (LSTM)               (None, 128)               66560     
                                                                 
 dense_3 (Dense)             (None, 10)                1290      
                                                                 
Total params: 67,850
Trainable params: 67,850
Non-trainable params: 0
_________________________________________________________________


In [15]:
print("Training ...")
batch_size = 45 # num of training examples per minibatch
num_epochs = 100
model.fit(
    X_train_mfcc,
    Y_one_hot_train,
    batch_size=batch_size,
    epochs=num_epochs,
)

Training ...
Epoch 1/100


2022-11-09 03:14:01.422608: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-11-09 03:14:02.347798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 105s 6s/step - loss: 2.2500 - accuracy: 0.1214
Epoch 2/100
16/16 [==============================] - 102s 6s/step - loss: 2.1007 - accuracy: 0.2043
Epoch 3/100
16/16 [==============================] - 101s 6s/step - loss: 2.0456 - accuracy: 0.2443
Epoch 4/100
16/16 [==============================] - 102s 6s/step - loss: 1.9688 - accuracy: 0.2529
Epoch 5/100
16/16 [==============================] - 104s 7s/step - loss: 1.9361 - accuracy: 0.2857
Epoch 6/100
16/16 [==============================] - 101s 6s/step - loss: 1.8991 - accuracy: 0.3114
Epoch 7/100
16/16 [==============================] - 100s 6s/step - loss: 1.8472 - accuracy: 0.3271
Epoch 8/100
16/16 [==============================] - 102s 6s/step - loss: 1.8341 - accuracy: 0.3371
Epoch 9/100
16/16 [==============================] - 103s 6s/step - loss: 1.8023 - accuracy: 0.3343
Epoch 10/100
16/16 [==============================] - 100s 6s/step - loss: 1.7758 - accuracy: 0.3600
Epoch 11/10

In [16]:
intermediate_layer_model = Model(inputs=model.input,
                                outputs=model.layers[1].output)


In [17]:
intermediate_output_train = intermediate_layer_model(X_train_mfcc)
intermediate_output_test = intermediate_layer_model(X_test_mfcc)

In [18]:
intermediate_output_train

<tf.Tensor: shape=(700, 128), dtype=float32, numpy=
array([[-0.3359743 , -0.46362358, -0.34727016, ...,  0.10481954,
        -0.34225914,  0.3109142 ],
       [-0.57793427, -0.48364237, -0.6238312 , ...,  0.08349159,
        -0.35375476,  0.43667102],
       [ 0.14967065, -0.7540211 , -0.2973293 , ...,  0.05854022,
        -0.26546338,  0.2438011 ],
       ...,
       [ 0.06634715,  0.0732023 ,  0.17573348, ...,  0.30999398,
        -0.6505117 ,  0.2662749 ],
       [-0.5882483 , -0.12079439, -0.12608057, ..., -0.04981716,
         0.08654279, -0.22126392],
       [-0.81755304, -0.11777642, -0.80095035, ...,  0.614314  ,
        -0.3023164 , -0.48225915]], dtype=float32)>

In [19]:
sknn.SVM_score(intermediate_output_train, Y_train_mfcc, intermediate_output_test, Y_test_mfcc)

0.44666666666666666

In [148]:
test_loss, test_acc = model.evaluate(X_test_zcr, Y_test_zcr, verbose=2)

2022-11-03 22:04:54.423506: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-03 22:04:54.478340: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 0s - loss: 1.0753 - accuracy: 0.1767 - 447ms/epoch - 45ms/step


In [21]:
test_loss, test_acc = model.evaluate(X_test_mfcc, Y_one_hot_test, verbose=2)

2022-11-09 14:31:26.849565: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 4s - loss: 1.5619 - accuracy: 0.4600 - 4s/epoch - 383ms/step


In [ ]:
test_loss, test_acc = model.evaluate(X_test_zcr_mfcc, Y_test_zcr_mfcc, verbose=2)

2022-11-04 03:45:55.908381: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-04 03:45:56.003917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-04 03:45:56.101725: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 - 1s - loss: 0.8142 - accuracy: 0.5967 - 943ms/epoch - 94ms/step


: 